Creando un Jupyter solo para las migrciones a SQL

In [2]:
import pandas as pd

In [12]:
# Lo primero es abrir los CSVs de cada API.

# Empezamos con la extracción de Spoti, que es la que contiene todos los datos, ya que para las otras APIs
# se han usado los artistas que arrojaba Spoti.

# Abrir dataframe de spoty para luego hacer la importacion. 
dataframe_spoty_csv = pd.read_csv('../data/dataframes_combinados.csv')

In [13]:
dataframe_spoty_csv

,nombre_cancion_album,artista,tipo_album_cancion,año_lanzamiento,género
0,Seventeen,Ladytron,album,2022,Electrónica
1,Crust,Flying Lotus,album,2021,Electrónica
2,Abu Simbel,WhoMadeWho,single,2021,Electrónica
3,Apricots,BICEP,album,2021,Electrónica
4,KILL DEM,Jamie xx,single,2022,Electrónica
...,...,...,...,...,...
3995,Raised Up Right,Drew Parker,single,2022,Rock
3996,Red Moon,Big Thief,album,2022,Rock
3997,Maybe You Saved Me,Bad Suns,album,2022,Rock
3998,homeswitcher,Jane Remover,single,2021,Rock


In [14]:
''' añadimos una columna que vaya autoincrementándose que será el id de la canción y la ponemos
al principio del df'''

dataframe_spoty_csv.insert(0, 'canción_id', range(1, len(dataframe_spoty_csv) + 1))


In [16]:
dataframe_spoty_csv

,canción_id,nombre_cancion_album,artista,tipo_album_cancion,año_lanzamiento,género
0,1,Seventeen,Ladytron,album,2022,Electrónica
1,2,Crust,Flying Lotus,album,2021,Electrónica
2,3,Abu Simbel,WhoMadeWho,single,2021,Electrónica
3,4,Apricots,BICEP,album,2021,Electrónica
4,5,KILL DEM,Jamie xx,single,2022,Electrónica
...,...,...,...,...,...,...
3995,3996,Raised Up Right,Drew Parker,single,2022,Rock
3996,3997,Red Moon,Big Thief,album,2022,Rock
3997,3998,Maybe You Saved Me,Bad Suns,album,2022,Rock
3998,3999,homeswitcher,Jane Remover,single,2021,Rock


In [19]:
# asignar un ID autoincremental a cada artista único y luego 
# reutilizar ese ID para las canciones del mismo artista.

# Crear el diccionario para almacenar el ID de cada artista
artista_id_map = {}
current_id = 1

# Función para asignar ID al artista
def asignar_id_artista(artista):
    global current_id
    if artista not in artista_id_map:
        artista_id_map[artista] = current_id
        current_id += 1
    return artista_id_map[artista]

# Aplicar la función a cada fila y crear la columna 'artista_id'
dataframe_spoty_csv['artista_id'] = dataframe_spoty_csv['artista'].apply(asignar_id_artista)

print(dataframe_spoty_csv)

      canción_id nombre_cancion_album            artista tipo_album_cancion  \
0              1            Seventeen           Ladytron              album   
1              2                Crust       Flying Lotus              album   
2              3           Abu Simbel         WhoMadeWho             single   
3              4             Apricots              BICEP              album   
4              5             KILL DEM           Jamie xx             single   
...          ...                  ...                ...                ...   
3995        3996      Raised Up Right        Drew Parker             single   
3996        3997             Red Moon          Big Thief              album   
3997        3998   Maybe You Saved Me           Bad Suns              album   
3998        3999         homeswitcher       Jane Remover             single   
3999        4000           Summertime  The Hip Abduction              album   

      año_lanzamiento       género  artista_id  
0 

In [ ]:
# Comprobando que está bien viendo todas las filas del df.
pd.set_option('display.max_rows', None)
# Esto vuelve a activar la opción de que no se vean todas las columnas.
pd.reset_option('display.max_rows')
dataframe_spoty_csv

,canción_id,nombre_cancion_album,artista,tipo_album_cancion,año_lanzamiento,género,artista_id
0,1,Seventeen,Ladytron,album,2022,Electrónica,1
1,2,Crust,Flying Lotus,album,2021,Electrónica,2
2,3,Abu Simbel,WhoMadeWho,single,2021,Electrónica,3
3,4,Apricots,BICEP,album,2021,Electrónica,4
4,5,KILL DEM,Jamie xx,single,2022,Electrónica,5
...,...,...,...,...,...,...,...
3995,3996,Raised Up Right,Drew Parker,single,2022,Rock,824
3996,3997,Red Moon,Big Thief,album,2022,Rock,576
3997,3998,Maybe You Saved Me,Bad Suns,album,2022,Rock,738
3998,3999,homeswitcher,Jane Remover,single,2021,Rock,869


In [6]:
# Vamos a probar a importar el df a SQL.
# Hemos creado la tabla en SQL con las columnas canción_id, nombre_cancion_album, artista_id,
# tipo_album_cancion, año_lanzamiento, género.

from sqlalchemy import create_engine

In [28]:
df_spoty_sin_artista = dataframe_spoty_csv[['canción_id', 'nombre_cancion_album', 'artista_id', 'tipo_album_cancion', 'año_lanzamiento', 'género']]

In [29]:
df_spoty_sin_artista

,canción_id,nombre_cancion_album,artista_id,tipo_album_cancion,año_lanzamiento,género
0,1,Seventeen,1,album,2022,Electrónica
1,2,Crust,2,album,2021,Electrónica
2,3,Abu Simbel,3,single,2021,Electrónica
3,4,Apricots,4,album,2021,Electrónica
4,5,KILL DEM,5,single,2022,Electrónica
...,...,...,...,...,...,...
3995,3996,Raised Up Right,824,single,2022,Rock
3996,3997,Red Moon,576,album,2022,Rock
3997,3998,Maybe You Saved Me,738,album,2022,Rock
3998,3999,homeswitcher,869,single,2021,Rock


In [7]:
user='root'
password='AlumnaAdalab'
host='127.0.0.1'
database= 'MusicStream_PR2_G2'
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(connection_string)

In [33]:
df_spoty_sin_artista.to_sql(name='Spotify', con=engine, if_exists='append', index=False)

4000

In [36]:
# Vamos a hacer lo mismo para crear una tabla que contenga artistas con sus IDs, que en ppio. 
# contendrá artistas duplicados ya que viene la tabla anterior con las canciones y sus artistas.

# creamos un df nuevo con las columnas que queremos y ponemos el drop_duplicates para que elimine los duplicados.

df_artistas_spoty = df_spoty_sin_artista = dataframe_spoty_csv[['artista', 'artista_id']].drop_duplicates()

In [37]:
df_artistas_spoty.to_sql(name='Artistas', con=engine, if_exists='append', index=False)
# No deja porque artista_id es PK. Habrá que hacer una copia de la tabla de Spoti

899

In [ ]:
#añadir dataframe de MB1 para ver si soluciona los artistas con distinto nombre
dataframe_MB1_csv = pd.read_csv('../data/dataframe_MB1.csv')
dataframe_MB1_csv.to_sql(name='musicbrainz1', con=engine, if_exists='append', index=False)

-1